In [1]:
pip install .

Processing /home/jovyan
  Created wheel for edacolors: filename=edacolors-0.0.1-py3-none-any.whl size=3484 sha256=611e4dd9e77898272dde4471c2b10b15e00a7a96cb1166b22ecd55cb05f80b79
  Stored in directory: /tmp/pip-ephem-wheel-cache-kvxxoa_w/wheels/fc/c4/49/78b5bd16ca276f2916d0829d47c131046b6e4575f7dd51e987
Successfully built edacolors
  Attempting uninstall: edacolors
    Found existing installation: edacolors 0.0.1
    Uninstalling edacolors-0.0.1:
      Successfully uninstalled edacolors-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install cmcrameri

Note: you may need to restart the kernel to use updated packages.


In [3]:
## Load packages 

# General packages
import numpy as np
import sys
import pandas as pd

# Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import random
import string

# eda colors
from edacolors import eda_themes
from edacolors.eda_themes import *
from cmcrameri import cm

# Load ApTViz visualization theme
pio.templates.default = "edacolors"

# Plotly helper (from here https://github.com/plotly/plotly.py/issues/2189)
def mpl_to_plotly(cmap, pl_entries=11, rdigits=2):
    # cmap - colormap 
    # pl_entries - int = number of Plotly colorscale entries
    # rdigits - int -=number of digits for rounding scale values
    scale = np.linspace(0, 1, pl_entries)
    colors = (cmap(scale)[:, :3]*255).astype(np.uint8)
    pl_colorscale = [[round(s, rdigits), f'rgb{tuple(color)}'] for s, color in zip(scale, colors)]
    return pl_colorscale

In [4]:
## Choose parameters for data and colormap

# Parameters
NSAMPLES = 1000
NCOLORS = 8   # MUST be <= 8 for ito, tol
GRADIENTMAP = cm.tokyo  # cm.lapaz, cm.batlow_r, cm...


In [5]:
## Generate data
# Note - assuming no negative data.

# Create different types of variables
cat1_vars = random.sample(string.ascii_lowercase,NCOLORS)
cat2_vars = random.sample(string.ascii_uppercase,NCOLORS)

cat1 = [random.choice(cat1_vars) for i in np.arange(NSAMPLES)]
cat2 = [random.choice(cat2_vars) for i in np.arange(NSAMPLES)]
binary1 = [random.choice([0, 1]) for i in np.arange(NSAMPLES)]
binary2 = [random.choice([0, 1]) for i in np.arange(NSAMPLES)]
ordinal1 = [random.choice(np.arange(NCOLORS)) for i in np.arange(NSAMPLES)]
ordinal2 = [random.choice(np.arange(NCOLORS)) for i in np.arange(NSAMPLES)]
cont1 = np.abs([random.gauss(5,2) for i in np.arange(NSAMPLES)])
cont2 = np.abs([random.gauss(5,2) for i in np.arange(NSAMPLES)])
cont3 = np.abs([random.gauss(5,2) for i in np.arange(NSAMPLES)])

df = pd.DataFrame({"cat1": cat1, "cat2": cat2, "binary1": binary1, "binary2": binary2,
                          "ordinal1": ordinal1, "ordinal2": ordinal2, 
                          "cont1": cont1, "cont2": cont2, "cont3": cont3})

# Add a little shift to continuous variables so they don't sit all on top of each other
for (i,var) in enumerate(cat1_vars):
    df.cont3[df.cat1 == var] += 5*i


df.head()

<ipython-input-5-f94bbb393e2c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.cont3[df.cat1 == var] += 5*i


,cat1,cat2,binary1,binary2,ordinal1,ordinal2,cont1,cont2,cont3
0,v,H,0,0,0,0,8.193933,1.882985,24.887427
1,b,H,1,1,6,3,5.686693,6.311827,20.221667
2,y,E,0,1,0,7,6.946739,7.692488,38.164313
3,e,U,1,1,6,0,1.466477,5.137683,16.125287
4,b,L,1,0,4,7,5.316913,6.863822,19.109242


In [6]:
## Convert map to plotly colormap

colormap_ptly = mpl_to_plotly(GRADIENTMAP, 255)

In [21]:
# Heatmap

heatmap_data = np.random.rand(20,20)
heatmap_data = heatmap_data + np.transpose(heatmap_data)
fig = go.Figure(data=go.Heatmap(
                    z=heatmap_data,
                    colorscale=colormap_ptly,
                    xgap=3,
                    ygap=3))
fig.update_xaxes(side="top", visible=False, showgrid= False,zeroline= False) # thick line at x=0)
fig.show()

In [13]:
# Scatter - overlay
# Note I've also altered marker opacity

fig = go.Figure(data=go.Scatter(
    x = df.cont1,
    y = df.cont2,
    mode='markers',
    marker=dict(
        size=16,
        color=df.cont3, #set color equal to a variable
        colorscale=colormap_ptly, # one of plotly colorscales
        showscale=True,
        opacity = 0.8
    )
))

fig.show()

In [9]:
# Histogram - overlay


fig = px.histogram(df,x='cont3', color='cat1', color_discrete_sequence=QUALITATIVEMAP, 
                  category_orders={"cat1": cat1_vars})
fig.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.8)
fig.show()

NameError: name 'QUALITATIVEMAP' is not defined

In [ ]:
# Histogram - stacked


fig = px.histogram(df,x='cont3', color='cat1', color_discrete_sequence=QUALITATIVEMAP, 
                  category_orders={"cat1": cat1_vars})
fig.update_layout(barmode='stack')

fig.show()

In [ ]:
# Box

fig = px.box(df, x="binary1", y="cont1", color="ordinal1", color_discrete_sequence=colormap_ptly)
fig.show()